### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1
    FEATURE_THRESHOLD = 1e-7

    def __init__(
        self,
        min_samples_split=2,
        max_depth=None,
        sufficient_share=1.0,
        criterion='gini',
        max_features=None
    ):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_y = lambda y: 1 - np.sum((np.bincount(y) / y.size) ** 2)
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_y = lambda y: - np.sum(np.bincount(y) / y.size * 
                                          np.log(np.bincount(y) / y.size))
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_y = lambda y: 1 - np.max(np.bincount(y) / y.size)
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (l_c**2 / l_s + r_c**2 / r_s).sum(axis=1) / (l_s + r_s)[0]

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(l_c * np.log(
            l_c / l_s
        ) + r_c * np.log(
            r_c / r_s
        )).sum(axis=1) / (l_s + r_s)[0]

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1-(np.max(l_c, axis=1) + np.max(r_c, axis=1)) / (l_s + r_s)[0]

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: int(np.sqrt(feature_ids.shape[0]))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: int(np.log(feature_ids.shape[0]) / np.log(2))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        feature_ids = self.get_feature_ids(x.shape[1])
        min_gain = np.inf
        for idx in feature_ids:
            col = x[:, idx]
            x_sort, y_sort = self.__sort_samples(col, y)

            # получаем те индексы, где класс отличается от предыдущего
            borders = np.where(y_sort[:-1] != y_sort[1:])[0] + 1

            # считаем, сколько идёт подряд классов до смены
            counts_in_row = borders - np.append(np.array([0]), borders[:-1])

            # splits - матрица, где количество строк равно
            # количеству смен классов
            # а количество столбцов - количеству классов
            # в каждой строке записываем количество подряд идущих
            # элементов класса в соответствующем индексе
            splits = np.zeros((borders.shape[0], self.num_class))
            splits[np.arange(borders.shape[0]), y_sort[borders - 1]] = 1
            splits = splits * counts_in_row.reshape(-1, 1)

            # left_split - матрица, где в каждой строке
            # количество элементов в левом поддереве,
            # right_splits - аналогично
            left_split = np.cumsum(splits, axis=0)
            right_split = np.bincount(
                y_sort,
                minlength=self.num_class
            ) - left_split

            left_size = left_split.sum(axis=1).reshape(-1, 1).astype(np.int32)
            right_size = y_sort.shape[0] - left_size

            gains = self.G_function(
                left_split,
                left_size,
                right_split,
                right_size)

            opt_gain = np.min(gains)
            idx_opt_gain = np.argmin(gains)
            left_el_idx = left_size[idx_opt_gain][0]
            if opt_gain < min_gain:
                res_idx = idx
                res_thr = (x_sort[left_el_idx - 1] + x_sort[left_el_idx]) / 2.0
                min_gain = opt_gain

        self.feature_importances_[res_idx] += x.shape[0] * (self.G_y(y) - min_gain)
        return res_idx, res_thr

    def __fit_node(self, x, y, node_id, depth):
        if depth == self.max_depth or \
        self.min_samples_split >= x.shape[0] or \
        len(np.unique(y)) == 1:
            tmp = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  tmp.argmax(), tmp / y.size)
            return

        feature_id, threshold = self.__find_threshold(x, y)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE,
                              feature_id, threshold)

        X_l, X_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)
        if X_l.shape[0] == 0 or X_r.shape[0] == 0:
            buf = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, buf.argmax(),
                                  buf.astype(float) / y.size)
            return

        self.__fit_node(X_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(X_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1]).astype(np.float32)
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= x.shape[0]

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 7.1 ms, sys: 140 µs, total: 7.24 ms
Wall time: 4.99 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 50 ms, sys: 3.66 ms, total: 53.7 ms
Wall time: 52.2 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.94074074074074077

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [9]:
from sklearn import preprocessing

In [10]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', sep=',', encoding="ISO-8859-1")
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
                 'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
                  'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'dec_o', 'attr_o',
                  'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
                  'prob_o', 'met_o', 'field', 'undergra',
                  'imprelig', 'imprace',
                  'from', 'zipcode', 'career', 'sports',
                  'tvsports', 'exercise',
                  'dining', 'museums',
                  'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
                  'theater', 'movies', 'concerts', 'music', 'shopping',
                  'yoga', 'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
pd.get_dummies(df, columns=['race'], prefix='race', prefix_sep='=')
df.loc[:, 'mn_sat'] =\
df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] =\
df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df.loc[:, 'income'] =\
df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'temp_totalsum'] =\
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                   'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
               'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                         'fun1_1', 'amb1_1', 'shar1_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                            'fun2_1', 'amb2_1', 'shar2_1']]\
                                   .sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
               'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                         'fun2_1', 'amb2_1', 'shar2_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [11]:
clf = DecisionTreeClassifier()
my_clf = MyDecisionTreeClassifier()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Проверка скорости работы на Speed Dating Data 

In [13]:
%time clf.fit(X_train, y_train)

CPU times: user 80.7 ms, sys: 7.65 ms, total: 88.3 ms
Wall time: 86.2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
%time my_clf.fit(X_train, y_train)

CPU times: user 540 ms, sys: 0 ns, total: 540 ms
Wall time: 540 ms


## Проверка качества работы на Speed Dating Data

In [15]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.55803289205000006

In [16]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5619407420221596

## Задание 3

In [18]:
df_pair.columns[np.argsort(-clf.feature_importances_)][: 10]

Index(['match', 'intel2_1_f', 'mn_sat_f', 'race', 'sinc1_1', 'race_f',
       'amb1_1_f', 'go_out', 'intel1_1_f', 'attr1_1_f'],
      dtype='object')

In [19]:
df_pair.columns[np.argsort(-my_clf.feature_importances_)][: 10]

Index(['match', 'shar2_1', 'attr1_1', 'exphappy', 'goal', 'field_cd',
       'intel1_1_f', 'amb2_1', 'race', 'samerace'],
      dtype='object')

## Задание 4

## Задание 5

In [22]:
forest = RandomForestClassifier()

grid = {
    "n_estimators": [10, 50, 100],
    "max_depth": [10, 15, 20],
    "criterion": ['gini', 'entropy'],
    "min_samples_leaf": [2, 4, 6]
}


search = RandomizedSearchCV(forest, param_distributions=grid)
search.fit(X_train, y_train)
search.best_params_

{'criterion': 'gini',
 'max_depth': 15,
 'min_samples_leaf': 4,
 'n_estimators': 10}